## Innebygginger

I vårt forrige eksempel jobbet vi med høydimensjonale bag-of-words-vektorer med lengde `vocab_size`, og vi konverterte eksplisitt fra lavdimensjonale posisjonsrepresentasjonsvektorer til sparsomme one-hot-representasjoner. Denne one-hot-representasjonen er ikke minneeffektiv, og i tillegg behandles hvert ord uavhengig av hverandre, dvs. one-hot-kodede vektorer uttrykker ingen semantisk likhet mellom ord.

I denne enheten skal vi fortsette å utforske **News AG**-datasettet. For å begynne, la oss laste inn dataene og hente noen definisjoner fra den forrige notatboken.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Hva er embedding?

Ideen med **embedding** er å representere ord ved lavdimensjonale tette vektorer, som på en eller annen måte reflekterer den semantiske betydningen av et ord. Senere skal vi diskutere hvordan man bygger meningsfulle ord-embeddings, men for nå kan vi bare tenke på embeddings som en måte å redusere dimensjonaliteten til en ordvektor. 

Så, en embedding-lag vil ta et ord som input og produsere en output-vektor med spesifisert `embedding_size`. På en måte er det veldig likt `Linear`-laget, men i stedet for å ta en én-hot kodet vektor, vil det kunne ta et ordnummer som input.

Ved å bruke embedding-laget som det første laget i vårt nettverk, kan vi gå fra bag-of-words til **embedding bag**-modellen, hvor vi først konverterer hvert ord i teksten vår til tilsvarende embedding, og deretter beregner en aggregatfunksjon over alle disse embeddingene, som for eksempel `sum`, `average` eller `max`.  

![Bilde som viser en embedding-klassifiserer for fem sekvensord.](../../../../../translated_images/no/embedding-classifier-example.b77f021a7ee67eee.webp)

Vårt klassifiserings-nevrale nettverk vil starte med embedding-lag, deretter et aggregasjonslag, og en lineær klassifiserer på toppen av det:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Håndtering av variabel sekvensstørrelse

Som et resultat av denne arkitekturen må minibatcher til nettverket vårt opprettes på en bestemt måte. I den forrige enheten, når vi brukte bag-of-words, hadde alle BoW-tensorer i en minibatch lik størrelse `vocab_size`, uavhengig av den faktiske lengden på tekstsekvensen vår. Når vi går over til ord-embedding, vil vi ende opp med et variabelt antall ord i hver tekstprøve, og når vi kombinerer disse prøvene til minibatcher, må vi legge til noe padding.

Dette kan gjøres ved å bruke samme teknikk som å gi `collate_fn`-funksjonen til datakilden:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Trene embedding-klassifiserer

Nå som vi har definert en passende dataloader, kan vi trene modellen ved å bruke treningsfunksjonen vi har definert i forrige enhet:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Merk**: Vi trener kun for 25k poster her (mindre enn én full epoke) for tidsbesparelse, men du kan fortsette treningen, skrive en funksjon for å trene over flere epoker, og eksperimentere med læringsrateparameteren for å oppnå høyere nøyaktighet. Du bør kunne oppnå en nøyaktighet på rundt 90%.


### EmbeddingBag-lag og representasjon av sekvenser med variabel lengde

I den tidligere arkitekturen måtte vi fylle alle sekvenser til samme lengde for å passe dem inn i en minibatch. Dette er ikke den mest effektive måten å representere sekvenser med variabel lengde på - en annen tilnærming ville være å bruke en **offset**-vektor, som holder offsetene til alle sekvenser lagret i én stor vektor.

![Bilde som viser en offset-sekvensrepresentasjon](../../../../../translated_images/no/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: På bildet ovenfor viser vi en sekvens av tegn, men i vårt eksempel jobber vi med sekvenser av ord. Prinsippet for å representere sekvenser med en offset-vektor forblir imidlertid det samme.

For å arbeide med offset-representasjon bruker vi [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html)-laget. Det ligner på `Embedding`, men det tar innhold-vektor og offset-vektor som input, og inkluderer også et gjennomsnittslag, som kan være `mean`, `sum` eller `max`.

Her er et modifisert nettverk som bruker `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

For å klargjøre datasettet for trening, må vi gi en konverteringsfunksjon som vil klargjøre offset-vektoren:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Merk at i motsetning til alle tidligere eksempler, aksepterer nettverket vårt nå to parametere: datavektor og offsetvektor, som har forskjellige størrelser. På samme måte gir datalasteren vår oss også 3 verdier i stedet for 2: både tekst- og offsetvektorer blir gitt som funksjoner. Derfor må vi justere treningsfunksjonen vår litt for å ta hensyn til dette:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantiske Embeddinger: Word2Vec

I vårt forrige eksempel lærte modellens embedding-lag å kartlegge ord til vektorrepresentasjoner, men denne representasjonen hadde ikke mye semantisk mening. Det hadde vært fint å lære en slik vektorrepresentasjon der lignende ord eller synonymer tilsvarer vektorer som er nær hverandre i henhold til en eller annen vektordistanse (f.eks. euklidisk distanse).

For å oppnå dette må vi forhåndstrene vår embedding-modell på en stor samling tekst på en spesifikk måte. En av de første metodene for å trene semantiske embeddinger kalles [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Den er basert på to hovedarkitekturer som brukes for å produsere en distribuert representasjon av ord:

 - **Continuous bag-of-words** (CBoW) — i denne arkitekturen trener vi modellen til å forutsi et ord basert på den omkringliggende konteksten. Gitt ngrammet $(W_{-2},W_{-1},W_0,W_1,W_2)$, er målet for modellen å forutsi $W_0$ ut fra $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** er motsatt av CBoW. Modellen bruker det omkringliggende vinduet av kontekstord for å forutsi det nåværende ordet.

CBoW er raskere, mens skip-gram er tregere, men gjør en bedre jobb med å representere sjeldne ord.

![Bilde som viser både CBoW- og Skip-Gram-algoritmer for å konvertere ord til vektorer.](../../../../../translated_images/no/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

For å eksperimentere med word2vec-embedding forhåndstrent på Google News-datasettet, kan vi bruke **gensim**-biblioteket. Nedenfor finner vi ordene som ligner mest på 'neural'

> **Merk:** Når du først oppretter ordvektorer, kan nedlastingen ta litt tid!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Vi kan også beregne vektorinnbygginger fra ordet, som skal brukes i treningsklassifiseringsmodellen (vi viser kun de første 20 komponentene av vektoren for klarhet):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Både CBoW og Skip-Grams er "prediktive" embeddinger, ettersom de kun tar lokale kontekster i betraktning. Word2Vec utnytter ikke global kontekst.

**FastText** bygger videre på Word2Vec ved å lære vektorrepresentasjoner for hvert ord og tegn-n-grammene som finnes i hvert ord. Verdiene av representasjonene blir deretter gjennomsnittet til én vektor i hvert treningssteg. Selv om dette legger til mye ekstra beregning under forhåndstrening, gjør det at ordembeddinger kan kode informasjon på sub-ordnivå.

En annen metode, **GloVe**, utnytter ideen om samforekomstmatriser og bruker nevrale metoder for å dekomponere samforekomstmatrisen til mer uttrykksfulle og ikke-lineære ordvektorer.

Du kan eksperimentere med eksempelet ved å bytte embeddinger til FastText og GloVe, siden gensim støtter flere forskjellige modeller for ordembedding.


## Bruke forhåndstrente embeddings i PyTorch

Vi kan endre eksemplet ovenfor for å forhåndsfylle matrisen i vår embedding-lag med semantiske embeddings, som Word2Vec. Vi må ta hensyn til at vokabularet til forhåndstrente embeddings og tekstkorpuset vårt sannsynligvis ikke vil samsvare, så vi vil initialisere vektene for de manglende ordene med tilfeldige verdier:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

I vårt tilfelle ser vi ikke en stor økning i nøyaktighet, noe som sannsynligvis skyldes ganske forskjellige ordforråd.  
For å løse problemet med ulike ordforråd, kan vi bruke en av følgende løsninger:  
* Tren word2vec-modellen på nytt med vårt ordforråd  
* Last inn datasettet vårt med ordforrådet fra den forhåndstrente word2vec-modellen. Ordforrådet som brukes til å laste inn datasettet kan spesifiseres under innlasting.  

Den sistnevnte tilnærmingen virker enklere, spesielt fordi PyTorch `torchtext`-rammeverket inneholder innebygd støtte for innebygginger. Vi kan for eksempel opprette et GloVe-basert ordforråd på følgende måte:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Lastet ordforråd har følgende grunnleggende operasjoner:
* `vocab.stoi`-ordbok lar oss konvertere et ord til dets ordbokindeks
* `vocab.itos` gjør det motsatte - konverterer et tall til et ord
* `vocab.vectors` er arrayet av innebyggingsvektorer, så for å få innebyggingen av et ord `s` må vi bruke `vocab.vectors[vocab.stoi[s]]`

Her er et eksempel på manipulering av innebygginger for å demonstrere ligningen **kind-man+woman = queen** (jeg måtte justere koeffisienten litt for å få det til å fungere):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

For å trene klassifisereren ved hjelp av disse embeddingene, må vi først kode datasettet vårt ved hjelp av GloVe-ordforråd:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Som vi har sett ovenfor, lagres alle vektorinbeddinger i `vocab.vectors`-matrisen. Det gjør det superenkelt å laste inn disse vektene i vektene til innbeddingslaget ved hjelp av enkel kopiering:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

En av grunnene til at vi ikke ser en betydelig økning i nøyaktighet er på grunn av at noen ord fra datasettet vårt mangler i den forhåndstrente GloVe-ordforrådet, og derfor blir de i hovedsak ignorert. For å overvinne dette kan vi trene våre egne innebygginger på datasettet vårt.


## Kontekstuelle Embeddinger

En viktig begrensning med tradisjonelle forhåndstrente embedding-representasjoner som Word2Vec er problemet med ordsanse-diskriminering. Selv om forhåndstrente embeddinger kan fange opp noe av betydningen av ord i kontekst, blir alle mulige betydninger av et ord kodet inn i samme embedding. Dette kan skape problemer i modeller som bruker disse embeddingene, siden mange ord, som ordet 'play', har forskjellige betydninger avhengig av konteksten de brukes i.

For eksempel har ordet 'play' i disse to setningene ganske forskjellige betydninger:
- Jeg dro på et **skuespill** på teateret.
- John vil **leke** med vennene sine.

De forhåndstrente embeddingene ovenfor representerer begge disse betydningene av ordet 'play' i samme embedding. For å overkomme denne begrensningen, må vi bygge embeddinger basert på **språkmodeller**, som er trent på et stort tekstkorpus og *vet* hvordan ord kan settes sammen i ulike kontekster. Å diskutere kontekstuelle embeddinger er utenfor rammen for denne opplæringen, men vi vil komme tilbake til dem når vi snakker om språkmodeller i neste enhet.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
